In [1]:
# ruff: noqa: E402
import math
import warnings
from typing import Dict, Literal

warnings.simplefilter("ignore")
import delu  # Deep Learning Utilities: https://github.com/Yura52/delu
import numpy as np
import pandas as pd
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.std import tqdm

warnings.resetwarnings()

from rtdl_revisiting_models import MLP, ResNet, FTTransformer

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Set random seeds in all libraries.
delu.random.seed(0)

0

## Dataset

In [3]:
# >>> Dataset.
TaskType = Literal["regression", "binclass", "multiclass"]

task_type: TaskType = "regression"
n_classes = None
dataset = pd.read_csv('/home/mattyshen/interpretableDistillation/simulations/Tabular_Gemstone_Price/data/train.csv').drop(columns =['id'])
X: np.ndarray = dataset.drop(columns = ['price']) #.to_numpy()
Y: np.ndarray = dataset['price'].to_numpy()

# >>> Continuous features.
X_cont: np.ndarray = X[['carat', 'depth', 'table', 'x', 'y', 'z']].to_numpy().astype(np.float32)
n_cont_features = X_cont.shape[1]

# >>> Categorical features.
# NOTE: the above datasets do not have categorical features, but,
# for the demonstration purposes, it is possible to generate them.
X_cat = X[['cut', 'color']]
for column in X_cat.columns:
    label_encoder = sklearn.preprocessing.LabelEncoder()
    X_cat[column] = label_encoder.fit_transform(X_cat[column]).astype(np.int64)
    
cat_cardinalities = [len(X_cat[c].value_counts()) for c in X_cat.columns]
X_cat = X_cat.to_numpy().astype(np.int64)

# >>> Labels.
# Regression labels must be represented by float32.
if task_type == "regression":
    Y = Y.astype(np.float32)
else:
    assert n_classes is not None
    Y = Y.astype(np.int64)
    assert set(Y.tolist()) == set(
        range(n_classes)
    ), "Classification labels must form the range [0, 1, ..., n_classes - 1]"

# >>> Split the dataset.
all_idx = np.arange(len(Y))
trainval_idx, test_idx = sklearn.model_selection.train_test_split(
    all_idx, train_size=0.8, random_state = 0
)
train_idx, val_idx = sklearn.model_selection.train_test_split(
    trainval_idx, train_size=0.8, random_state = 0
)
data_numpy = {
    "train": {"x_cont": X_cont[train_idx], "y": Y[train_idx]},
    "val": {"x_cont": X_cont[val_idx], "y": Y[val_idx]},
    "test": {"x_cont": X_cont[test_idx], "y": Y[test_idx]},
}
if X_cat is not None:
    data_numpy["train"]["x_cat"] = X_cat[train_idx]
    data_numpy["val"]["x_cat"] = X_cat[val_idx]
    data_numpy["test"]["x_cat"] = X_cat[test_idx]

/tmp/ipykernel_6529/432957109.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat[column] = label_encoder.fit_transform(X_cat[column]).astype(np.int64)
/tmp/ipykernel_6529/432957109.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat[column] = label_encoder.fit_transform(X_cat[column]).astype(np.int64)


## Preprocessing

In [4]:
# >>> Feature preprocessing.
# NOTE
# The choice between preprocessing strategies depends on a task and a model.

# (A) Simple preprocessing strategy.
# preprocessing = sklearn.preprocessing.StandardScaler().fit(
#     data_numpy['train']['x_cont']
# )

# (B) Fancy preprocessing strategy.
# The noise is added to improve the output of QuantileTransformer in some cases.
X_cont_train_numpy = data_numpy["train"]["x_cont"]
noise = (
    np.random.default_rng(0)
    .normal(0.0, 1e-5, X_cont_train_numpy.shape)
    .astype(X_cont_train_numpy.dtype)
)
preprocessing = sklearn.preprocessing.QuantileTransformer(
    n_quantiles=max(min(len(train_idx) // 30, 1000), 10),
    output_distribution="normal",
    subsample=10**9,
).fit(X_cont_train_numpy + noise)
del X_cont_train_numpy

for part in data_numpy:
    data_numpy[part]["x_cont"] = preprocessing.transform(data_numpy[part]["x_cont"])

# >>> Label preprocessing.
if task_type == "regression":
    Y_mean = data_numpy["train"]["y"].mean().item()
    Y_std = data_numpy["train"]["y"].std().item()
    for part in data_numpy:
        data_numpy[part]["y"] = (data_numpy[part]["y"] - Y_mean) / Y_std

# >>> Convert data to tensors.
data = {
    part: {k: torch.as_tensor(v, device=device) for k, v in data_numpy[part].items()}
    for part in data_numpy
}

if X_cat is not None:
    data["train"]["x_cat"] = data["train"]["x_cat"].to(torch.int64)
    data["val"]["x_cat"] = data["val"]["x_cat"].to(torch.int64)
    data["test"]["x_cat"] = data["test"]["x_cat"].to(torch.int64)

if task_type != "multiclass":
    # Required by F.binary_cross_entropy_with_logits
    for part in data:
        data[part]["y"] = data[part]["y"].float()

## Model

In [ ]:
# The output size.
d_out = n_classes if task_type == "multiclass" else 1

# # NOTE: uncomment to train MLP
mlp = MLP(
    d_in=n_cont_features + sum(cat_cardinalities),
    d_out=d_out,
    n_blocks=2,
    d_block=384,
    dropout=0.1,
).to(device)
mlp_optim = torch.optim.AdamW(mlp.parameters(), lr=3e-4, weight_decay=1e-5)

# # NOTE: uncomment to train ResNet
res = ResNet(
    d_in=n_cont_features + sum(cat_cardinalities),
    d_out=d_out,
    n_blocks=2,
    d_block=192,
    d_hidden=None,
    d_hidden_multiplier=2.0,
    dropout1=0.3,
    dropout2=0.0,
).to(device)
res_optim = torch.optim.AdamW(res.parameters(), lr=3e-4, weight_decay=1e-5)

ftt = FTTransformer(
    n_cont_features=n_cont_features,
    cat_cardinalities=cat_cardinalities,
    d_out=d_out,
    **FTTransformer.get_default_kwargs(),
).to(device)
ftt_optim = ftt.make_default_optimizer()

models = [mlp, res, ftt]
optimizers = [mlp_optim, res_optim, ftt_optim]

## Training

In [ ]:
def apply_model(model, batch: Dict[str, Tensor]) -> Tensor:
    if isinstance(model, (MLP, ResNet)):
        x_cat_ohe = (
            [
                F.one_hot(column, cardinality)
                for column, cardinality in zip(batch["x_cat"].T, cat_cardinalities)
            ]
            if "x_cat" in batch
            else []
        )
        return model(torch.column_stack([batch["x_cont"]] + x_cat_ohe)).squeeze(-1)

    elif isinstance(model, FTTransformer):
        return model(batch["x_cont"], batch.get("x_cat")).squeeze(-1)

    else:
        raise RuntimeError(f"Unknown model type: {type(model)}")


loss_fn = (
    F.binary_cross_entropy_with_logits
    if task_type == "binclass"
    else F.cross_entropy
    if task_type == "multiclass"
    else F.mse_loss
)


@torch.no_grad()
def evaluate(model, part: str, ret=False) -> float:
    model.eval()

    eval_batch_size = 8096
    y_pred = (
        torch.cat(
            [
                apply_model(model, batch)
                for batch in delu.iter_batches(data[part], eval_batch_size)
            ]
        )
        .cpu()
        .numpy()
    )
    y_true = data[part]["y"].cpu().numpy()
    if ret:
        return y_pred

    if task_type == "binclass":
        y_pred = np.round(scipy.special.expit(y_pred))
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
    elif task_type == "multiclass":
        y_pred = y_pred.argmax(1)
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
    else:
        assert task_type == "regression"
        score = -(sklearn.metrics.mean_squared_error(y_true, y_pred) ** 0.5 * Y_std)
    return score  # The higher -- the better.


#print(f'Test score before training: {evaluate("test"):.4f}')
print(f'MLP val score before training: {evaluate(mlp, "val"):.4f}')
print(f'ResNet val score before training: {evaluate(res, "val"):.4f}')
print(f'FTTransformer val score before training: {evaluate(ftt, "val"):.4f}')

In [ ]:
data['train']['x_cont'].shape

# For demonstration purposes (fast training and bad performance),
# one can set smaller values:
# n_epochs = 20
# patience = 2
model_names = ['mlp', 'resnet', 'fttransformer']
n_epochs = 10
patience = 16

batch_size = 25000
epoch_size = math.ceil(len(train_idx) / batch_size)
timer = delu.tools.Timer()
early_stoppings = [delu.tools.EarlyStopping(patience, mode="max"), delu.tools.EarlyStopping(patience, mode="max"), delu.tools.EarlyStopping(patience, mode="max")]
best = [{"val": -math.inf,"epoch": -1,}, {"val": -math.inf,"epoch": -1,}, {"val": -math.inf,"epoch": -1,}]

print(f"Device: {device.type.upper()}")
print("-" * 88 + "\n")
timer.run()
for epoch in range(n_epochs):
    for batch in tqdm(
        delu.iter_batches(data["train"], batch_size, shuffle=True),
        desc=f"Epoch {epoch}",
        total=epoch_size,
    ):
        for model, optimizer in zip(models, optimizers):
            model.train()
            optimizer.zero_grad()
            loss = loss_fn(apply_model(model, batch), batch["y"])
            loss.backward()
            optimizer.step()
    for i, (model, e_s) in enumerate(zip(models, early_stoppings)):
        val_score = evaluate(model, "val")
        print(f"(val) {val_score:.4f} [time] {timer}")

        e_s.update(val_score)
        if e_s.should_stop():
            break

        if val_score > best[i]["val"]:
            #print("🌸 New best epoch! 🌸")
            best[i] = {"val": val_score, "epoch": epoch}
            torch.save(model.state_dict(), f'models/{model_names[i]}_best_val')
        print()

print("\n\nResult:")
print(best)

In [ ]:
defaults = [MLP(
    d_in=n_cont_features + sum(cat_cardinalities),
    d_out=d_out,
    n_blocks=2,
    d_block=384,
    dropout=0.1,
), ResNet(
    d_in=n_cont_features + sum(cat_cardinalities),
    d_out=d_out,
    n_blocks=2,
    d_block=192,
    d_hidden=None,
    d_hidden_multiplier=2.0,
    dropout1=0.3,
    dropout2=0.0,
), FTTransformer(
    n_cont_features=n_cont_features,
    cat_cardinalities=cat_cardinalities,
    d_out=d_out,
    **FTTransformer.get_default_kwargs(),
)]

In [ ]:
import json
for i, model in enumerate(models):
    model_class = defaults[i]
    model_class.load_state_dict(torch.load(f'models/{model_names[i]}_best_val'))
    model_class = model_class.to(device)
    model_data_preds = data_numpy.copy()
    for k in model_data_preds.keys():
        model_data_preds[k]['y_hat'] = evaluate(model_class, k, ret=True)
        
    for k in model_data_preds.keys():
        for j in model_data_preds[k].keys():
            if type(model_data_preds[k][j]) != type([2, 4]):
                model_data_preds[k][j] = model_data_preds[k][j].tolist()
            
    with open(f'predictions/{model_names[i]}_data_predictions.json', 'w') as fp:
        json.dump(model_data_preds, fp)

In [20]:
pwd

'/home/mattyshen/interpretableDistillation'

In [5]:
import json

with open('/home/mattyshen/interpretableDistillation/simulations/Tabular_Gemstone_Price/predictions/mlp_binary_data_predictions.json', 'r') as openfile:
    mlp_dict = json.load(openfile)
    
with open('/home/mattyshen/interpretableDistillation/simulations/Tabular_Gemstone_Price/predictions/resnet_binary_data_predictions.json', 'r') as openfile:
    res_dict = json.load(openfile)
    
with open('/home/mattyshen/interpretableDistillation/simulations/Tabular_Gemstone_Price/predictions/fttransformer_binary_data_predictions.json', 'r') as openfile:
    tff_dict = json.load(openfile)

In [6]:
import os

os.chdir('/home/mattyshen/interpretableDistillation/')

from interpretDistill.fourierDistill import FTDistill, FTDistillCV
from interpretDistill.binaryTransformer import BinaryTransformer

In [14]:
#ftd_val = FTDistillCV(selection = 'L1L2', lam1_range=np.array([0.1, 1, 10]), lam2_range=np.array([0.1, 1, 10]), size_interactions = 2)#$
ftd_val = FTDistill(selection = 'L1L2', lam1 = 0.01, lam2 = 0.01)

In [9]:
X_train = X.iloc[train_idx, :]
y_train = pd.Series(tff_dict['train']['y_hat'], name = 'price')

X_val = X.iloc[val_idx, :]
y_val = pd.Series(tff_dict['val']['y_hat'], name = 'price')

X_test = X.iloc[test_idx, :]
y_test = pd.Series(tff_dict['test']['y_hat'], name = 'price')

In [16]:
bt_val = BinaryTransformer(depth=1, bit = False)
#X_val_bv = bt_val.fit_and_transform(X_val, y_val)
X_train_bv = bt_val.fit_and_transform(X_train, pd.Series(tff_dict['train']['y'], name = 'price'))
X_val_bv = bt_val.transform(X_val)
# X_val = bt.fit_and_transform(X_val, y_val)
# X_train = bt.transform(X_train)
X_test_bv = bt_val.transform(X_test)

In [17]:
ftd_val.fit(X_val_bv, y_val)

poly fitted
regression model fitting
(30972, 41449)


In [18]:
np.mean(ftd_val.regression_model.coef_ != 0), np.sum(ftd_val.regression_model.coef_ != 0)

(0.0015440662018384038, 64)

In [19]:
def filter_strings(strings, bool_array, weight):
    filtered_strings = []
    for string, boolean, w in zip(strings, bool_array, weight):
        if boolean:
            filtered_strings.append((string, w*Y_std + Y_mean))
    return filtered_strings

In [20]:
filter_strings(ftd_val.features, list(ftd_val.regression_model.coef_ != 0), ftd_val.regression_model.coef_)

[(('clarity_IF', 'cut_Fair'), 3964.122284620108),
 (('clarity_IF', 'cut_Fair', 'color_F'), 3964.407704374141),
 (('clarity_IF', 'cut_Fair', 'y_1'), 3966.338731236759),
 (('clarity_IF', 'cut_Fair', 'y_2'), 3962.6406334502244),
 (('clarity_IF', 'carat_2', 'cut_Fair', 'carat_1'), 3964.4168406199224),
 (('clarity_SI2', 'cut_Fair', 'cut_Premium', 'color_H'), 3960.0457229346666),
 (('cut_Very', 'cut_Fair', 'Good', 'color_D', 'color_H'), 3956.600537665268),
 (('color_J', 'cut_Fair', 'clarity_VS2', 'color_D'), 3964.1608317614478),
 (('clarity_IF', 'cut_Fair', 'clarity_VS1', 'color_D'), 3958.1835297845646),
 (('clarity_IF', 'cut_Fair', 'color_F', 'x_1'), 3964.260394582576),
 (('clarity_IF', 'cut_Fair', 'x_2', 'color_F'), 3964.310516618364),
 (('clarity_IF', 'cut_Fair', 'color_F', 'y_1'), 3964.1780706276736),
 (('clarity_IF', 'y_2', 'cut_Fair', 'color_F'), 3965.51982333822),
 (('clarity_IF', 'cut_Fair', 'depth_2', 'depth_1'), 3964.9770095274257),
 (('clarity_IF', 'cut_Fair', 'table_2', 'table_1'

In [21]:
mean_squared_error(ftd_val.predict(X_val_bv), y_val)

NameError: name 'mean_squared_error' is not defined

In [ ]:
mean_squared_error(ftd_val.predict(X_val_bv), y_val.to_numpy())

In [ ]:
mean_squared_error(np.array(mlp_dict['val']['y_hat'])*Y_std + Y_mean, y_val.to_numpy()*Y_std + Y_mean)

In [22]:
from sklearn.metrics import mean_squared_error
mse = pd.DataFrame()
mse['model'] = ['mlp', 'resent', 'fttransformer', 'ftdistill']
mse['train'] = [mean_squared_error(np.array(mlp_dict['train']['y'])*Y_std + Y_mean, np.array(mlp_dict['train']['y_hat'])*Y_std + Y_mean),
               mean_squared_error(np.array(res_dict['train']['y'])*Y_std + Y_mean, np.array(res_dict['train']['y_hat'])*Y_std + Y_mean),
               mean_squared_error(np.array(tff_dict['train']['y'])*Y_std + Y_mean, np.array(tff_dict['train']['y_hat'])*Y_std + Y_mean),
                mean_squared_error(ftd_val.predict(X_train_bv)*Y_std + Y_mean, np.array(tff_dict['train']['y_hat'])*Y_std + Y_mean)]
mse['val'] = [mean_squared_error(np.array(mlp_dict['val']['y'])*Y_std + Y_mean, np.array(mlp_dict['val']['y_hat'])*Y_std + Y_mean),
               mean_squared_error(np.array(res_dict['val']['y'])*Y_std + Y_mean, np.array(res_dict['val']['y_hat'])*Y_std + Y_mean),
               mean_squared_error(np.array(tff_dict['val']['y'])*Y_std + Y_mean, np.array(tff_dict['val']['y_hat'])*Y_std + Y_mean),
              mean_squared_error(ftd_val.predict(X_val_bv)*Y_std + Y_mean, np.array(tff_dict['val']['y_hat'])*Y_std + Y_mean)]
mse['test'] = [mean_squared_error(np.array(mlp_dict['test']['y'])*Y_std + Y_mean, np.array(mlp_dict['test']['y_hat'])*Y_std + Y_mean),
               mean_squared_error(np.array(res_dict['test']['y'])*Y_std + Y_mean, np.array(res_dict['test']['y_hat'])*Y_std + Y_mean),
               mean_squared_error(np.array(tff_dict['test']['y'])*Y_std + Y_mean, np.array(tff_dict['test']['y_hat'])*Y_std + Y_mean),
               mean_squared_error(ftd_val.predict(X_test_bv)*Y_std + Y_mean, np.array(tff_dict['test']['y_hat'])*Y_std + Y_mean)]
mse

/home/mattyshen/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


,model,train,val,test
0,mlp,4.881051e+06,4.890721e+06,4.965528e+06
1,resent,4.888593e+06,4.891446e+06,4.967253e+06
2,fttransformer,4.908055e+06,4.897881e+06,4.969716e+06
3,ftdistill,1.109209e+07,1.096270e+07,1.119337e+07


In [35]:
from sklearn.metrics import r2_score
r2 = pd.DataFrame()
r2['model'] = ['mlp', 'resent', 'fttransformer', 'ftdistill']
r2['train'] = [r2_score(np.array(mlp_dict['train']['y'])*Y_std + Y_mean, np.array(mlp_dict['train']['y_hat'])*Y_std + Y_mean),
               r2_score(np.array(res_dict['train']['y'])*Y_std + Y_mean, np.array(res_dict['train']['y_hat'])*Y_std + Y_mean),
               r2_score(np.array(tff_dict['train']['y'])*Y_std + Y_mean, np.array(tff_dict['train']['y_hat'])*Y_std + Y_mean),
              r2_score(ftd_val.predict(X_train_bv)*Y_std + Y_mean, np.array(tff_dict['train']['y_hat'])*Y_std + Y_mean)]
r2['val'] = [r2_score(np.array(mlp_dict['val']['y'])*Y_std + Y_mean, np.array(mlp_dict['val']['y_hat'])*Y_std + Y_mean),
               r2_score(np.array(res_dict['val']['y'])*Y_std + Y_mean, np.array(res_dict['val']['y_hat'])*Y_std + Y_mean),
               r2_score(np.array(tff_dict['val']['y'])*Y_std + Y_mean, np.array(tff_dict['val']['y_hat'])*Y_std + Y_mean),
            r2_score(ftd_val.predict(X_val_bv)*Y_std + Y_mean, np.array(tff_dict['val']['y_hat'])*Y_std + Y_mean)]
r2['test'] = [r2_score(np.array(mlp_dict['test']['y'])*Y_std + Y_mean, np.array(mlp_dict['test']['y_hat'])*Y_std + Y_mean),
               r2_score(np.array(res_dict['test']['y'])*Y_std + Y_mean, np.array(res_dict['test']['y_hat'])*Y_std + Y_mean),
               r2_score(np.array(tff_dict['test']['y'])*Y_std + Y_mean, np.array(tff_dict['test']['y_hat'])*Y_std + Y_mean),
                r2_score(ftd_val.predict(X_test_bv)*Y_std + Y_mean, np.array(tff_dict['test']['y_hat'])*Y_std + Y_mean)]
r2

/home/mattyshen/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


,model,train,val,test
0,mlp,0.699325,0.696151,0.696386
1,resent,0.698860,0.696106,0.696280
2,fttransformer,0.697661,0.695707,0.696130
3,ftdistill,-8423.180429,-8224.389672,-8449.435473
